In [1]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.

import pandas as pd
# Make the graphs a bit prettier, and bigger
pd.set_option('display.mpl_style', 'default')

# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 20)

import numpy as np
import math
# The usual preamble
%matplotlib inline
%pylab inline
import matplotlib.pyplot as plt
plt.rcParams['axes.color_cycle'] = ['r', 'g', 'b', 'c']
plt.rcParams['lines.color'] = 'r'
plt.rcParams['figure.figsize'] = (15, 5)

import process
from modeling import categorize

Populating the interactive namespace from numpy and matplotlib


/Users/along528/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
import pickle
#clf = pickle.load(open('randomforestclassifier.p','rb'))
best_feature_names = pickle.load(open('best_features_names.p','rb'))
model = pickle.load(open('randomforestclassifier_bad.p','rb'))
print model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=15, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [71]:
df_with_rpsi = process.add_features(process.get_data(munge=True,with_traffic=True))
df_all = process.get_data(munge=False,with_traffic=False)
df_all_munge = process.add_features(process.get_data(munge=True,with_traffic=False))
#df_all = df_all_munge.merge(df_all,how='left') #in case dropped some during munging
df_all = df_all.merge(pd.DataFrame(df_with_rpsi['rpsi']),how='left',right_index=True,left_index=True)
df_all['is_measured'] = np.isnan(df_all['rpsi'])==False


In [72]:
val_df,_ = process.get_split_add_data()
processor = process.Processor(val_df[best_feature_names])
X_predict,_ = processor.get_scaled_Xy(df_all_munge[best_feature_names])
y_predict = model.predict(X_predict)
df_all_predictions = pd.DataFrame(y_predict,columns=['rpsi_predicted'],index=df_all_munge.index)
df_all = df_all.merge(df_all_predictions,how='left',right_index=True,left_index=True)
df_all = df_all.fillna(-1)
df_all = df_all[((df_all['rpsi']==-1) & (df_all['rpsi_predicted']==-1))==False]


,agency,city,state,zipcode,agcytype,swnauthemp,swnftemp,swnptemp,civftemp,civptemp,totftemp,totptemp,ftreserveswn,ptreserveswn,ftreserveciv,ptreserveciv,ftgangoff,ptgangoff,ftdrugoff,ptdrugoff,ftterroff,pterroff,fthumtrfoff,pthumtrfoff,numrespoff,numcpo,numsro,numpatr,numinvst,numjail,...,institutionalized_adult_state_prison_all,institutionalized_adult_local_jail_all,institutionalized_juvenile_all,institutionalized_white,institutionalized_adult_white,institutionalized_adult_federal_detention_white,institutionalized_adult_federal_prison_white,institutionalized_adult_state_prison_white,institutionalized_adult_local_jail_white,institutionalized_juvenile_white,institutionalized_black,institutionalized_adult_black,institutionalized_adult_federal_detention_black,institutionalized_adult_federal_prison_black,institutionalized_adult_state_prison_black,institutionalized_adult_local_jail_black,institutionalized_juvenile_black,population_white,population_black,population_native_american,population_asian,population_nathaw,population_other_race,population_mult_race,total_income_estimate_all,total_income_estimate_white,total_income_estimate_black,rpsi,is_measured,rpsi_predicted
surveyid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Anchorage Police Department,Anchorage,AK,99507,3,399,385,0,174,0,559,0,0,0,0,0,6,0,12,0,6,0,7,0,182,6,18,182,76,0,...,0,629,190,1264,434,0,0,0,277,74,166,99,0,0,0,70,11,151441,14039,21549,22442,5724,6154,20688,88288,63011,5501,-1,False,1
1001,Ketchikan Police Department,Ketchikan,AK,99901,3,23,23,0,10,1,33,1,0,0,0,0,0,0,0,0,0,0,0,0,23,0,2,,,,...,69,0,30,57,30,0,0,30,0,14,4,0,0,0,0,0,3,9204,79,1912,943,27,93,1250,5305,4025,23,-1,False,1
16,Tuscaloosa Police Department,Tuscaloosa,AL,35401,3,269,264,0,75,0,339,0,0,0,0,0,0,0,11,0,1,0,0,0,157,7,10,157,58,0,...,0,615,68,463,246,0,0,0,246,37,471,366,0,0,0,366,29,62416,45082,262,1883,23,1767,1221,39364,22646,15603,-1,False,1
13,Mobile Police Department,Mobile,AL,36606,3,532,526,0,254,96,780,96,0,6,0,0,0,0,5,0,1,0,0,0,304,30,5,345,97,0,...,0,1320,131,1441,457,0,0,0,457,41,1330,856,0,0,0,856,90,136887,121541,948,4901,141,2453,3841,104285,57283,43754,-1,False,1
1004,Anniston Police Department,Anniston,AL,36202,3,92,81,0,33,0,114,0,0,0,0,0,0,0,1,0,0,0,0,0,69,0,0,,,,...,0,467,77,761,268,0,0,0,268,49,283,194,0,0,0,194,27,48199,18934,282,618,28,1618,1139,27258,18677,7732,-1,False,1
5,Birmingham Police Department,Birmingham,AL,35203,3,914,768,0,400,0,1168,0,0,0,0,0,4,0,0,0,1,0,0,0,462,462,12,528,137,1,...,280,1111,258,2232,501,0,0,173,328,75,1911,854,0,0,107,747,178,248229,216662,1109,10220,194,11592,5800,200127,105638,85888,-1,False,1
15,Montgomery Police Department,Montgomery,AL,36101,3,510,491,0,97,0,588,0,0,0,0,0,2,0,1,0,2,0,0,0,212,12,15,212,88,2,...,2004,509,305,2117,1481,51,408,921,101,103,3027,2265,274,504,1081,406,200,81206,120864,535,4643,166,4545,2744,82722,34348,45439,-1,False,1
7,Dothan Police Department,Dothan,AL,36303,3,157,148,0,100,0,248,0,0,0,0,15,0,0,0,0,0,0,0,0,80,0,12,80,38,22,...,0,439,46,700,238,0,0,0,188,21,466,314,0,0,0,251,25,53580,23271,297,764,45,1071,1418,31759,22364,8524,-1,False,1
1018,Frisco City Police Department,Frisco City,AL,36445,3,4,3,2,0,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,,,,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3522,1323,71,11,5,24,94,1846,1297,506,-1,False,0


In [76]:
df_all.to_sql('app_db',process.engine,if_exists='replace')

In [77]:
df_all.to_csv('app_db.csv')